In [ ]:
import sys
import math
import os

In [ ]:
def SItoSoE(parameters, Print_To_File, Files_Location, Is_Top, modules):

    N = parameters["N"]
    HRR = parameters["HRR"]
    Enable_Input_Latch = parameters["ENABLE_INPUT_LATCH"]
    Enable_Output_Registers = parameters["ENABLE_SIM_OUTPUT_REGISTERS"]

    #IN_WIDTH>=1
    IN_WIDTH = 10

    NOEPS = math.ceil(N/HRR)
    while math.ceil(N/NOEPS) != HRR:
        HRR -= 1

    ModuleName="SItoSoE_{}_HRx{}_".format(N, HRR)
    if Enable_Input_Latch<=0:
        ModuleName+="N"
    ModuleName+="IL_"
    if Enable_Output_Registers<=0:
        ModuleName+="N"
    ModuleName+="OR"
    
    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)
    
    if Print_To_File<=0:
        of=sys.stdout
    else:
        of=open(Files_Location+ModuleName+".v", 'w+')

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("output reg newInSeriesStart = 1,", file=of)
    lghrr=math.ceil(math.log2(HRR))
    if lghrr==1:
        print("output reg inSeries = 0,", file=of)
    elif lghrr>1:
        print("output reg [{}:0] inSeries = 0,".format(lghrr-1), file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("A{}, ".format(i), end='', file=of)
    print(file=of)
    NE = N - NOEPS*(HRR-1)
    while NE<=0:
        NE += NOEPS
    print("output ", end='', file=of)
    if Enable_Output_Registers>0:
        print("reg ", end='', file=of)
    if NE==1:
        print("O0OutReady",  end='', file=of)
    else:
        print("O0toO{}OutReady".format(NE-1),  end='', file=of)
    if Enable_Output_Registers>0:
        print(" = 0,", file=of)
    else:
        print(",",file=of)
    if NE==NOEPS:
        print("output reg ONOutReady=0, //not used", file=of)
    else:
        print("output ", end='', file=of)
        if Enable_Output_Registers>0:
            print("reg ", end='', file=of)
        if NE==NOEPS-1:
            print("O{}OutReady".format(NE), end='', file=of)
        else:
            print("O{}toO{}OutReady".format(NE, NOEPS-1), end='', file=of)
        if Enable_Output_Registers>0:
            print(" = 0,", file=of)
        else:
            print(",", file=of)
    print("output ", end='', file=of)
    if Enable_Output_Registers>0:
        print("reg ", end='', file=of)
    if lghrr==1:
        print("outSeries", end='', file=of)
    elif lghrr>1:
        print("[{}:0] outSeries".format(lghrr-1),  end='', file=of)
    if Enable_Output_Registers>0:
        print(" = {},".format(HRR-1), file=of)
    else:
        print(",", file=of)
    print("output reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NOEPS):
        print("O{}, ".format(i), end='', file=of)
    print(file=of)
    print("output ", end='', file=of)
    if Enable_Output_Registers<=0:
        print("reg ", end='', file=of)
    if NE==1:
        print("O0earlyOutReady", end='', file=of)
    else:
        print("O0toO{}earlyOutReady".format(NE-1), end='', file=of)
    if Enable_Output_Registers<=0:
        print(" = 0, //not used", file=of)
    else:
        print(",", file=of)
    if NE==NOEPS:
        print("output reg ONearlyOutReady=0 //not used", file=of)
    else:
        print("output ", end='', file=of)
        if Enable_Output_Registers<=0:
            print("reg ", end='', file=of)
        if NE==NOEPS-1:
            print("O{}earlyOutReady".format(NE), end='', file=of)
        else:
            print("O{}toO{}earlyOutReady".format(NE, NOEPS-1), end='', file=of)
        if Enable_Output_Registers<=0:
            print(" = 0 //not used", file=of)
        else:
            print(file=of)
    print(");\n", file=of)

    #(inSeries==0) ? inReady : 1
    if NE==1:
        print("assign O0", end='', file=of)
    else:
        print("assign O0toO{}".format(NE-1), end='', file=of)
    if Enable_Output_Registers>0:
        print("early", end='', file=of)
    print("OutReady = ", end='', file=of)
    if Enable_Input_Latch>0:
        print("((inSeries==0) & inReady) | (inSeries!=0);", file=of)
    else:
        print("inReady;", file=of)

    if NE!=NOEPS:
        if NE==NOEPS-1:
            print("assign O{}".format(NE), end='', file=of)
        else:
            print("assign O{}toO{}".format(NE, NOEPS-1), end='', file=of)
        if Enable_Output_Registers>0:
            print("early", end='', file=of)
        print("OutReady = ", end='', file=of)
        if HRR==2:
            print("(inSeries==0) & inReady;", file=of) #inSeries!=1
        else:
            if Enable_Input_Latch<=0:
                print("(inSeries!={}) & inReady;".format(HRR-1), file=of)
            else:
                #if HRR==3:
                #    print("((inSeries==0) & inReady) | (inSeries==1);".format(HRR-1), file=of)
                #else:
                    print("((inSeries==0) & inReady) | ((inSeries!=0) & (inSeries!={}));".format(HRR-1), file=of)

    # print("always @(posedge clk) begin", file=of)
    # print("\tif(reset) begin", file=of)
    # print("\t\tnewInSeriesStart <= 1;", file=of)
    # print("\t\tinSeries <= 0;", file=of)
    # print("\tend", file=of)
    # print("\telse if(enable", end='', file=of)
    # if Enable_Input_Latch<=0:
    #     print(" & inReady", end='', file=of)
    # print(") begin", file=of)
    # print("\t\tif(inSeries==0) begin", file=of)
    # if Enable_Input_Latch>0:
    #     print("\t\t\tif(inReady) begin", file=of)
    #     print("\t", end='', file=of)
    # print("\t\t\tnewInSeriesStart <= 0;", file=of)
    # if Enable_Input_Latch>0:
    #     print("\t", end='', file=of)
    # print("\t\t\tinSeries <= 1;", file=of)
    # if Enable_Input_Latch>0:
    #     print("\t\t\tend", file=of)
    # print("\t\tend", file=of)
    # print("\t\telse", end='', file=of)
    # if HRR>2:
    #     print(" if(inSeries=={})".format(HRR-1), end='', file=of)
    # print(" begin", file=of)
    # print("\t\t\tnewInSeriesStart <= 1;", file=of)
    # print("\t\t\tinSeries <= 0;", file=of)
    # print("\t\tend", file=of)
    # if HRR>2:
    #     print("\t\telse begin", file=of)
    #     if HRR==3:
    #         print("\t\t\tinSeries <= 2;", file=of)        
    #     else:
    #         print("\t\t\tinSeries <= inSeries+1;", file=of)
    #     print("\t\tend", file=of)
    # print("\tend", file=of)
    # print("end", file=of)
    # print(file=of)

    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tnewInSeriesStart <= 1;", file=of)
    print("\t\tinSeries <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable & ", end='', file=of)
    if Enable_Input_Latch<=0:
        print("inReady", end='', file=of)
    else:
        print("(inReady | (inSeries!=0))", end='', file=of)
    print(") begin", file=of)
    print("\t\tif(inSeries=={}) begin".format(HRR-1), file=of)
    print("\t\t\tnewInSeriesStart <= 1;", file=of)
    print("\t\t\tinSeries <= 0;", file=of)
    print("\t\tend", file=of)
    print("\t\telse begin", file=of)
    print("\t\t\tnewInSeriesStart <= 0;", file=of)
    if HRR==2:
        print("\t\t\tinSeries <= 1;", file=of)    
    else:
        print("\t\t\tinSeries <= inSeries+1;", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

    if Enable_Input_Latch>0:
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(NOEPS,N-1):
            print("A{}l, ".format(i), end='', file=of)
        print("A{}l;".format(N-1), file=of)
        print("always @(posedge clk) begin", file=of)
        print("\tif(enable) begin", file=of)
        print("\t\tif(inReady & (inSeries==0)) begin", file=of)
        print("\t\t\t", end='', file=of)
        for i in range(NOEPS,N):
            print("A{}l <= A{}; ".format(i, i), end='', file=of)
        print("\n\t\tend", file=of)
        print("\tend", file=of)
        print("end", file=of)
        print(file=of)

    if Enable_Output_Registers>0:
        print("always @(posedge clk) if(enable) begin", file=of)
        if NE==1:
            print("\tif(O0earlyOutReady) begin", file=of)
        else:
            print("\tif(O0toO{}earlyOutReady) begin".format(NE-1), file=of)
    else:
        print("always @(*) begin", file=of)
    print("\tcase(inSeries)", file=of)
    for i in range(HRR):
    #     if i==0:
    #         print("\t\t0: ", end='', file=of)
    #         if Enable_Output_Registers>0:
    #              print("if(inReady) ", end='', file=of)
    #         print("begin", file=of) #currently more power efficient, can use "begin" for simpler hardware
    #     else:
    #         print("\t\t{}: begin".format(i), file=of)
        print("\t\t{}: begin".format(i), file=of)
        print("\t\t\t", end='', file=of)
        for j in range(NOEPS):
            print("O{} ".format(j), end='', file=of)
            if Enable_Output_Registers>0:
                print("<", end='', file=of)
            k=NOEPS*i+j
            if k >= N:
                print("= 0; ", end='', file=of)
            elif k<NOEPS:
                print("= A{}; ".format(k), end='', file=of)
            else:
                print("= A{}".format(k), end='', file=of)
                if Enable_Input_Latch>0:
                    print("l", end='', file=of)
                print("; ", end='', file=of)
        print(file=of)
        print("\t\tend", file=of)
    if Enable_Output_Registers<=0 and (HRR & (HRR-1)) != 0: #check if HRR is not power of 2
        print("\t\tdefault: begin".format(k), file=of)
        print("\t\t\t", end='', file=of)
        for j in range(NOEPS):
            print("O{} = 0; ".format(j), end='', file=of)
        print(file=of)
        print("\t\tend", file=of)
    print("\tendcase", file=of)
    if Enable_Output_Registers>0:
        print("\tend", file=of)
        if NE==1:
            print("\tO0OutReady <= O0earlyOutReady;", file=of)
        else:
            print("\tO0toO{}OutReady <= O0toO{}earlyOutReady;".format(NE-1, NE-1), file=of)
        if NE==NOEPS-1:
            print("\tO{}OutReady <= O{}earlyOutReady;".format(NE, NE), file=of)
        elif NE!=NOEPS:
            print("\tO{}toO{}OutReady <= O{}toO{}earlyOutReady;".format(NE, NOEPS-1, NE, NOEPS-1), file=of)
    print("end\n", file=of)

    if Enable_Output_Registers<=0:
        print("assign outSeries = inSeries;", file=of)
    else:
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\toutSeries <= {};".format(HRR-1), file=of)
        print("\tend", file=of)
        print("\telse if(enable & ", end='', file=of)    
        if NE==1:
            print("O0earlyOutReady", end='', file=of)
        else:
            print("O0toO{}earlyOutReady".format(NE-1), end='', file=of)
        print(") begin", file=of)    
        #if HRR==2:
        #    print("\t\toutSeries <= ~outSeries;", file=of)
        #else:
        print("\t\tif(outSeries=={}) begin".format(HRR-1), file=of)
        print("\t\t\toutSeries <= 0;", file=of)
        print("\t\tend", file=of)
        print("\t\telse begin", file=of)
        if HRR==2:
            print("\t\t\toutSeries <= 1;", file=of)        
        else:
            print("\t\t\toutSeries <= outSeries+1;", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end", file=of)
    print(file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()
        
    modules[ModuleName]="SItoSoE"
    return ModuleName